<a href="https://colab.research.google.com/github/raiaiaia/NPL-PIBIC-CNPq/blob/master/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import nltk
import numpy as np
import pandas as pd
import csv
import re
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from unicodedata import normalize
import unicodedata


In [0]:
dataset = pd.read_csv ('/content/drive/My Drive/PROJETO/2020/classificados.csv')

dataset.head()


,RAISSA,RAYANNE,GETULIO,-1,0,1,Voto,Text
0,1.0,1.0,1.0,0,0,3,1,Ainda tô sem palavras pra essa união e pro dia...
1,0.0,0.0,0.0,0,3,0,0,#AFederalLuta\n#TiraAMãoDoNossoIF https://t.co...
2,1.0,1.0,1.0,0,0,3,1,É incrível ver a quantidade de estudante que a...
3,0.0,0.0,0.0,0,3,0,0,#AFederalLuta\n#HojeAAulaÉNaRua https://t.co/q...
4,1.0,1.0,1.0,0,0,3,1,Um amor por essa Foto #AFEDERALLUTA \n#TIRAAMA...


In [0]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RAISSA   4993 non-null   float64
 1   RAYANNE  4996 non-null   float64
 2   GETULIO  4995 non-null   float64
 3   -1       4999 non-null   int64  
 4   0        4999 non-null   int64  
 5   1        4999 non-null   int64  
 6   Voto     4999 non-null   int64  
 7   Text     4998 non-null   object 
dtypes: float64(3), int64(4), object(1)
memory usage: 312.6+ KB


In [0]:
dataset.columns

Index(['RAISSA', 'RAYANNE', 'GETULIO', '-1', '0', '1', 'Voto', 'Text'], dtype='object')

In [0]:
dataset.groupby(by='Voto').size()

Voto
-1     587
 0    2764
 1    1648
dtype: int64

In [0]:
dataset.head(10)

,RAISSA,RAYANNE,GETULIO,-1,0,1,Voto,Text
0,1.0,1.0,1.0,0,0,3,1,Ainda tô sem palavras pra essa união e pro dia...
1,0.0,0.0,0.0,0,3,0,0,#AFederalLuta\n#TiraAMãoDoNossoIF https://t.co...
2,1.0,1.0,1.0,0,0,3,1,É incrível ver a quantidade de estudante que a...
3,0.0,0.0,0.0,0,3,0,0,#AFederalLuta\n#HojeAAulaÉNaRua https://t.co/q...
4,1.0,1.0,1.0,0,0,3,1,Um amor por essa Foto #AFEDERALLUTA \n#TIRAAMA...
5,1.0,1.0,1.0,0,0,3,1,experiência incrível nessa manifestação de hoj...
6,0.0,0.0,0.0,0,3,0,0,#tiraamaodomeuif #afederalluta https://t.co/IR...
7,0.0,1.0,0.0,0,2,1,0,"NÃO É GASTO, É INVESTIMENTO \n#Tiraamaodomeuif..."
8,0.0,0.0,0.0,0,3,0,0,só verdades #tiramaodomeuif #todospelaeducacao...
9,1.0,1.0,0.0,0,1,2,1,Congresso Nacional: Em defesa das Universidade...


In [0]:
dataset['Text'] = dataset.Text.str.lower()
dataset.head()

,RAISSA,RAYANNE,GETULIO,-1,0,1,Voto,Text
0,1.0,1.0,1.0,0,0,3,1,ainda tô sem palavras pra essa união e pro dia...
1,0.0,0.0,0.0,0,3,0,0,#afederalluta\n#tiraamãodonossoif https://t.co...
2,1.0,1.0,1.0,0,0,3,1,é incrível ver a quantidade de estudante que a...
3,0.0,0.0,0.0,0,3,0,0,#afederalluta\n#hojeaaulaénarua https://t.co/q...
4,1.0,1.0,1.0,0,0,3,1,um amor por essa foto #afederalluta \n#tiraama...


In [0]:
def remove_hashtags(item):
  if '#' in item:
    return item.replace('#', '').replace('#', '')
  else:
      return item


dataset['Text'] = [remove_hashtags(str(t)) for t in dataset['Text']]

In [0]:
dataset.head(10)

,RAISSA,RAYANNE,GETULIO,-1,0,1,Voto,Text
0,1.0,1.0,1.0,0,0,3,1,ainda tô sem palavras pra essa união e pro dia...
1,0.0,0.0,0.0,0,3,0,0,afederalluta\ntiraamãodonossoif https://t.co/h...
2,1.0,1.0,1.0,0,0,3,1,é incrível ver a quantidade de estudante que a...
3,0.0,0.0,0.0,0,3,0,0,afederalluta\nhojeaaulaénarua https://t.co/qaz...
4,1.0,1.0,1.0,0,0,3,1,um amor por essa foto afederalluta \ntiraamaod...
5,1.0,1.0,1.0,0,0,3,1,experiência incrível nessa manifestação de hoj...
6,0.0,0.0,0.0,0,3,0,0,tiraamaodomeuif afederalluta https://t.co/irzr...
7,0.0,1.0,0.0,0,2,1,0,"não é gasto, é investimento \ntiraamaodomeuif ..."
8,0.0,0.0,0.0,0,3,0,0,só verdades tiramaodomeuif todospelaeducacao e...
9,1.0,1.0,0.0,0,1,2,1,congresso nacional: em defesa das universidade...


In [0]:
stopwords = ['pra', 'pro','to', 'ta','de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'a', 'com', 'nao', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'a', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'ha', 'nos', 'ja', 'esta', 'eu', 'tambam', 'sa3', 'pelo', 'pela', 'ata', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estao', 'vocaa', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'a s', 'minha', 'taam', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'sera', 'na3s', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocaas', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 
'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'esta', 'estamos', 'estao', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estavamos', 'estavam', 'estivera', 'estivaramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivassemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'ha', 'havemos', 'hao', 'houve', 'houvemos', 'houveram', 'houvera', 'houvaramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvassemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houvera', 'houveremos', 'houverao', 'houveria', 'houveraamos', 'houveriam', 'sou', 'somos', 'sao', 'era', 'aramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fa ramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fa ssemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'sera', 'seremos', 'serao', 'seria', 'seraamos', 'seriam', 'tenho', 'tem', 'temos', 'tam', 'tinha', 'tanhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tivaramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivassemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'tera', 'teremos', 'terao', 'teria', 'teraamos', 'teriam']
print(len(stopwords))

224


In [0]:
def remove_stopWords(sentence):
    frase = []
    for word in sentence.split():
        if word not in stopwords:
           # semStop = [p for p in word.split() if p not in stopwords]
            frase.append(word)
    return ' '.join(frase)

In [0]:
dataset['Text'] = [remove_stopWords(str(t)) for t in dataset['Text']]
dataset.head(10)

,RAISSA,RAYANNE,GETULIO,-1,0,1,Voto,Text
0,1.0,1.0,1.0,0,0,3,1,ainda tô palavras união dia hoje. lindo! ifc a...
1,0.0,0.0,0.0,0,3,0,0,afederalluta tiraamãodonossoif https://t.co/hp...
2,1.0,1.0,1.0,0,0,3,1,é incrível ver quantidade estudante ainda preo...
3,0.0,0.0,0.0,0,3,0,0,afederalluta hojeaaulaénarua https://t.co/qazb...
4,1.0,1.0,1.0,0,0,3,1,amor foto afederalluta tiraamaodomeuif https:/...
5,1.0,1.0,1.0,0,0,3,1,experiência incrível nessa manifestação hoje a...
6,0.0,0.0,0.0,0,3,0,0,tiraamaodomeuif afederalluta https://t.co/irzr...
7,0.0,1.0,0.0,0,2,1,0,"não é gasto, é investimento tiraamaodomeuif af..."
8,0.0,0.0,0.0,0,3,0,0,só verdades tiramaodomeuif todospelaeducacao e...
9,1.0,1.0,0.0,0,1,2,1,congresso nacional: defesa universidades públi...


In [0]:
dataset['Text'] = dataset['Text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

dataset.head(5000)

,RAISSA,RAYANNE,GETULIO,-1,0,1,Voto,Text
0,1.0,1.0,1.0,0,0,3,1,ainda tô palavras união dia hoje. lindo! ifc a...
1,0.0,0.0,0.0,0,3,0,0,afederalluta tiraamãodonossoif
2,1.0,1.0,1.0,0,0,3,1,é incrível ver quantidade estudante ainda preo...
3,0.0,0.0,0.0,0,3,0,0,afederalluta hojeaaulaénarua
4,1.0,1.0,1.0,0,0,3,1,amor foto afederalluta tiraamaodomeuif
...,...,...,...,...,...,...,...,...
4994,0.0,0.0,0.0,0,3,0,0,dia13érua tsunami13a
4995,0.0,0.0,0.0,0,3,0,0,énós tsunami13a dia13erua àsruaspovo lulalivre
4996,0.0,1.0,1.0,0,1,2,1,arrumando ir derrubar governo tsunami13a
4997,0.0,0.0,0.0,0,3,0,0,dce ufmg até presente momento não divulgou ond...


In [0]:
def removerAcentosECaracteresEspeciais(palavra):

    nfkd = unicodedata.normalize('NFKD', palavra)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    palavraSemAcento = re.sub('[^a-zA-Z0-9 \\\]', '', palavraSemAcento)

    return palavraSemAcento

In [0]:
dataset['Text'] = [removerAcentosECaracteresEspeciais(str(t)) for t in dataset['Text']]
dataset.head(5000)

,RAISSA,RAYANNE,GETULIO,-1,0,1,Voto,Text
0,1.0,1.0,1.0,0,0,3,1,ainda to palavras uniao dia hoje lindo ifc amo...
1,0.0,0.0,0.0,0,3,0,0,afederalluta tiraamaodonossoif
2,1.0,1.0,1.0,0,0,3,1,e incrivel ver quantidade estudante ainda preo...
3,0.0,0.0,0.0,0,3,0,0,afederalluta hojeaaulaenarua
4,1.0,1.0,1.0,0,0,3,1,amor foto afederalluta tiraamaodomeuif
...,...,...,...,...,...,...,...,...
4994,0.0,0.0,0.0,0,3,0,0,dia13erua tsunami13a
4995,0.0,0.0,0.0,0,3,0,0,enos tsunami13a dia13erua asruaspovo lulalivre
4996,0.0,1.0,1.0,0,1,2,1,arrumando ir derrubar governo tsunami13a
4997,0.0,0.0,0.0,0,3,0,0,dce ufmg ate presente momento nao divulgou ond...


In [0]:
dataset['Text'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
4994    False
4995    False
4996    False
4997    False
4998    False
Name: Text, Length: 4999, dtype: bool

In [0]:
tweets = dataset['Text'].values.astype('U')
classes = dataset['Voto'].values.astype('U')

vectorizer = CountVectorizer(analyzer = 'word')
freq_tweets = vectorizer.fit_transform(tweets)

In [0]:
features = list(dataset.columns[6:7])
x = dataset[features]
print(features)

['Voto']


In [0]:
target = list(dataset.columns[7:8])
y = dataset[target]
print(target)

['Text']


In [0]:
modelo = tree.DecisionTreeClassifier(criterion='gini')
modelo.fit(freq_tweets, classes)
#modelo = modelo.fit(x,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
positivo = 0;
negativo = 0;
neutro = 0;
pos = []
neg = []
neu = []

for dado in dataset['Voto']:

  if dado == 1:
   positivo += 1
   pos.append(dado)

  if dado == -1:
    negativo +=1
    neg.append(dado)

  if dado == 0:
    neutro +=1
    neu.append(dado)

In [0]:
print(negativo)
#print(neg)

587


In [0]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)

In [0]:
metrics.accuracy_score(classes, resultados)
#metrics.accuracy_score(x,y)

0.6095219043808762

In [0]:
sentimentos = ["Positivo", "Negativo", "Neutro"]

print(pd.crosstab(classes, resultados, rownames = ["Real"], colnames=["Predito"], margins=True))

Predito   -1     0     1   All
Real                          
-1        95   252   240   587
0        203  1921   640  2764
1        171   446  1031  1648
All      469  2619  1911  4999


In [0]:
print(metrics.classification_report(resultados, classes))

              precision    recall  f1-score   support

          -1       0.16      0.20      0.18       469
           0       0.70      0.73      0.71      2619
           1       0.63      0.54      0.58      1911

    accuracy                           0.61      4999
   macro avg       0.49      0.49      0.49      4999
weighted avg       0.62      0.61      0.61      4999

